In [1]:
import sys
sys.path.append('/content/drive/MyDrive/MobileCount/CrowdCounting-on-VisDrone2020-main/src')

In [2]:
"""
Script to compute latency and fps of a model
"""
import argparse
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from ast import literal_eval
import gc

import torch
import torch.backends.cudnn as cudnn
from torchvision.models import vgg16, vgg19, vgg11
from tabulate import tabulate
from config import cfg
from models.CC import CrowdCounter
from dataset.random import RandomDataset


def load_CC(MC):
    cc = CrowdCounter([0], MC)

    cc.load('/content/drive/MyDrive/MobileCount/CrowdCounting-on-VisDrone2020-main/exp/09-08_19-05_VisDrone_MobileCountx0_75_0.001__540x960_OD_CC_BS4/all_ep_1_mae_31.7_rmse_45.9.pth')


    return cc


def load_CC05(): return load_CC('MobileCountx0_5')
def load_CC075(): return load_CC('MobileCountx0_75')
def load_CC1(): return load_CC('MobileCount')
def load_CC125(): return load_CC('MobileCountx1_25')
def load_CC2(): return load_CC('MobileCountx2')


models = {'vgg16': vgg16,
          'vgg19': vgg19,
          'vgg11': vgg11,
          'CC05': load_CC05,
          'CC075': load_CC075,
          'CC1': load_CC1,
          'CC125': load_CC125,
          'CC2': load_CC2
          }


def measure_forward(model, x):
    """
    Measure the time for executing the the given tensor

    @param model: The model for measuring
    @param dataset: The tensor for measuring
    @return: Time elapsed for the execution
    """
    # synchronize gpu time and measure fp
    torch.cuda.synchronize()
    t0 = time.perf_counter()
    with torch.no_grad():
        y_pred = model.predict(x.cuda())
    torch.cuda.synchronize()
    elapsed_fp = time.perf_counter() - t0

    return elapsed_fp


def measure_fps(model, dataset):
    """
    Measure the time for executing the whole dataset

    @param model: The model for measuring
    @param dataset: The dataset for measuring
    @return: Time elapsed for the execution
    """
    # synchronize gpu time and measure fp
    torch.cuda.synchronize()
    t0 = time.perf_counter()
    with torch.no_grad():
        for x in dataset:
          _ = model.predict(x.cuda())
    torch.cuda.synchronize()
    elapsed_fp = time.perf_counter() - t0

    return elapsed_fp


def benchmark_forward(model, x, num_runs, num_warmup_runs):
    """
    Bench the forward time of the given model on the item

    @param model: the model to bench
    @param x: Tensor
    @param num_runs: number of runs to test
    @param num_warmup_runs: number of warmup forwards before every test
    @return: list of time for executing a forward
    """
    print('\nStarting warmup')
    # DRY RUNS
    for i in tqdm(range(num_warmup_runs)):
        _ = measure_forward(model, x)

    print('\nDone, now benchmarking')

    # START BENCHMARKING
    t_forward = []
    for x in tqdm(num_runs):
        t_fp = measure_forward(model, x)
        t_forward.append(t_fp)

    # free memory
    del model

    return t_forward


def benchmark_fps(model, dataset, num_runs, num_warmup_runs):
    """
    Bench the fps of the given model on the given dataset

    @param model: the model to bench
    @param dataset: DataLoader object
    @param num_runs: number of runs to test
    @param num_warmup_runs: number of warmup forwards before every test
    @return: list of time for executing a test
    """
    print('\nStarting warmup')
    # DRY RUNS
    x = None
    for it in dataset:
        x = it
        break

    for i in tqdm(range(num_warmup_runs)):
        _ = measure_forward(model, x)

    print('\nDone, now benchmarking')

    # START BENCHMARKING
    t_run = []
    for i in tqdm(range(num_runs)):
        t_fp = measure_fps(model, dataset)
        t_run.append(t_fp)

    # free memory
    del model

    return t_run


class Benchmarker:
    def __init__(self, model, dataset, batch_sizes, out_file, n_workers):
        """
        Initialize a Benchmarker object

        @param model: the Pytorch model to test
        @param dataset: Dataset object where to execute the test
        @param batch_sizes: Batch sizes list for executing the test
        @param out_file: output file for saving the results
        @param n_workers: number of workers for loading the dataset examples
        """
        torch.manual_seed(1234)
        cudnn.benchmark = True
        # transfer the model on GPU
        self.model = model.cuda().eval()
        self.dataset = dataset
        self.out_file = out_file
        self.batch_sizes = batch_sizes
        self.n_workers = n_workers

    def bench_forward(self, num_runs, num_warmup_runs):
        """
        Bench the forward time of the model averaging through num_runs, output the results on the stdout
        and eventually on file

        @param num_runs: the number of runs to execute
        @param num_warmup_runs: warmup forward before a run
        """
        mean_tfp = []
        std_tfp = []
        for i, bs in enumerate(self.batch_sizes):
            print('\nBatch size is: ' + str(bs))
            print('--------------------------')
            data_loader = torch.utils.data.DataLoader(
                self.dataset, batch_size=bs, shuffle=False, num_workers=self.n_workers
            )
            tmp = benchmark_forward(model, num_runs, num_warmup_runs, data_loader)
            # NOTE: we are estimating inference time per image
            mean_tfp.append(np.asarray(tmp).mean() / bs * 1e3)
            std_tfp.append(np.asarray(tmp).std() / bs * 1e3)

        self.out_results({'mean (ms)': mean_tfp, 'std (ms)': std_tfp})

        # force garbage collection
        gc.collect()

    def bench_fps(self, num_runs, num_warmup_runs):
        """
        Bench the fps count of the model averaging through num_runs, output the results on the stdout
        and eventually on file

        @param num_runs: the number of runs to execute
        @param num_warmup_runs: warmup forward before a run
        """
        mean_fps = []
        std_fps = []
        for i, bs in enumerate(self.batch_sizes):
            print('\nBatch size is: ' + str(bs))
            print('--------------------------')
            data_loader = torch.utils.data.DataLoader(
                self.dataset, batch_size=bs, shuffle=False, num_workers=self.n_workers
            )
            tmp = benchmark_fps(model, data_loader, num_runs, num_warmup_runs)
            mean_fps.append((1 / (np.asarray(tmp) / len(self.dataset))).mean())
            std_fps.append((1 / (np.asarray(tmp) / len(self.dataset))).std())

        self.out_results({'mean (fps)': mean_fps, 'std (fps)': std_fps})

        # force garbage collection
        gc.collect()

        return mean_fps, std_fps

    def out_results(self, dictionary):
        """
        Prints results on the stdout and on file if defined

        @param dictionary: dictionary of results
        """
        df = pd.DataFrame(dictionary, index=self.batch_sizes)
        #size = 'Input size: ' + str(self.dataset.shape())
        table = tabulate(df, headers='keys', tablefmt='psql')
        device = 'Device: ' + torch.cuda.get_device_name(torch.cuda.current_device())
        #result = "%s\n%s\n%s" % (device, size, table)
        result = "%s\n%s" % (device, table)


        print(result)
        if self.out_file != 'none':
            with open(self.out_file, 'a') as f:
                f.write(result)



In [ ]:
import glob
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, path, shape):
        self.imgs_path = path
        self.data = []
        self.shape = shape
        for img_path in glob.glob(self.imgs_path + "*.jpg"):
            self.data.append([img_path])
        print(self.data)
        #self.img_dim = (1920, 1080)
        self.img_dim = (960, 540)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path= self.data[idx]

        img = Image.open(img_path[0])
        trans = transforms.ToPILImage()
        trans1 = transforms.ToTensor()
        img_tensor = trans1(img)
        return img_tensor

    def shape(self):
      return self.shape


In [ ]:
import os

if __name__ == '__main__':
  
    model = models['CC075']()
    bs = literal_eval('[1,2,4]') #[1,2,4]
    #in_size = literal_eval('(3, 1080, 1920)')
    in_size = literal_eval('(3, 540, 960)')


    #path = '/content/drive/MyDrive/MobileCount/Generare_density/CrowdCounting/dataset_1920x1080/test/images/'
    path = '/content/drive/MyDrive/MobileCount/Generare_density/CrowdCounting/dataset_960x540/test/images/'
    
    #path = '/content/drive/MyDrive/MobileCount/CrowdCounting-on-VisDrone2020-main/src/images/'

    mean_list = []
    std_list = []

    for sequence in os.listdir(path):

      sequence_path = path+sequence+"/"
      dataset = CustomDataset(sequence_path,in_size)
      
      with open('/content/drive/MyDrive/MobileCount/Generare_density/CrowdCounting/dataset_960x540/test/results mobilecountx0.75 10-3 bs 4 finetuning.txt', 'a') as f:
        f.write("\nTest con sequenza: " + sequence +"\n")
      
      print("Test con sequenza: " + sequence)
      print('Model is loaded, start forwarding.')
      benchmarker = Benchmarker(model, dataset, bs, '/content/drive/MyDrive/MobileCount/Generare_density/CrowdCounting/dataset_960x540/test/results mobilecountx0.75 10-3 bs 4 finetuning.txt', 2)
      mean_fps, std_fps = benchmarker.bench_fps(105, 5)

      mean_list.append(mean_fps)
      std_list.append(std_fps)
    
    print('Media FPS: ' , str(np.mean(mean_list)))
    print('Std FPS: ' , str(np.mean(std_list)))



In [ ]:
#PRINT PREDICTION

In [4]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 8.4 MB/s 


In [2]:
import h5py
import torch
with h5py.File('/content/drive/MyDrive/MobileCount/Generare_density/CrowdCounting/dataset_1920x1080/test/images/img112/img112012.h5', 'r') as hf:
	gt = hf.get('density')[:]

print(gt)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
#PREDICTION
from main import run_net

run_net("/content/drive/MyDrive/MobileCount/Generare_density/CrowdCounting/dataset_1920x1080/test/to_print", ["display_callback"])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
'''if __name__ == '__main__':
  
    model = models['CC1']()
    bs = literal_eval('[4]')
    in_size = literal_eval('(3, 512, 512)')


    dataset = RandomDataset(in_size, 1)

    print(dataset.shape())

    print('Model is loaded, start forwarding.')
    benchmarker = Benchmarker(model, dataset, bs, '/content/results/results.txt', 2)
    benchmarker.bench_fps(105, 5)
'''

"if __name__ == '__main__':\n  \n    model = models['CC1']()\n    bs = literal_eval('[4]')\n    in_size = literal_eval('(3, 512, 512)')\n\n\n    dataset = RandomDataset(in_size, 1)\n\n    print(dataset.shape())\n\n    print('Model is loaded, start forwarding.')\n    benchmarker = Benchmarker(model, dataset, bs, '/content/results/results.txt', 2)\n    benchmarker.bench_fps(105, 5)\n"